In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import os
import boto3
import botocore
import re
from sagemaker import get_execution_role
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# initialise S3 information

role = get_execution_role()
region = boto3.Session().region_name

bucket='sagemaker-cert-bucket' # put your s3 bucket name here, and create s3 bucket
prefix = 'sagemaker/tensorflow-regression-abalone'
# customize to your bucket where you have stored the data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

# download the dataset from S3

BUCKET_NAME = 'sagemaker-cert-bucket' # replace with your bucket name
KEY = 'abalone/abalone.csv' # replace with your object key

s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, 'abalone.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise
        
# prepare the data

data = pd.read_csv("abalone.csv", header=0)
column_names = ['SEX', 'LEN', 'DIA', 'HEI', 'W1', 'W2', 'W3', 'W4', 'RIN']

data.columns = column_names

data = pd.get_dummies(data)

labels = data["RIN"]
data = data.drop(['RIN'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# upload data

X_train.to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

X_test.to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket, prefix), content_type='csv')


inputs = {'train': s3_input_train,
          'test': s3_input_test}

train_instance_type = 'ml.c4.xlarge'
hyperparameters = {'epochs': 500, 'batch_size': 32}

estimator = TensorFlow(entry_point='script.py',
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       base_job_name='tf-eager-scriptmode-abalone',
                       framework_version='1.12.0',
                       py_version='py3')

estimator.fit(inputs)

predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')
results = predictor.predict(x_test[:10])['predictions'] 